In [20]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum
import pandas as pd
import numpy as np
import os

In [27]:
model = LpProblem(name="profit-optimization", sense=LpMaximize)

In [28]:
# 读取Excel文件
df1 = pd.read_excel('/home/maxdmx/math-China/附件1(1).xlsx')
column_name_1 = '地块名称'
column_data_1 = df1[column_name_1].tolist()

df2 = pd.read_excel('/home/maxdmx/math-China/附件1(2).xlsx')
column_name_2 = '作物名称'
column_data_2 =df2[column_name_2].tolist()
column_data_2

['黄豆',
 '黑豆',
 '红豆',
 '绿豆',
 '爬豆',
 '小麦',
 '玉米',
 '谷子',
 '高粱',
 '黍子',
 '荞麦',
 '南瓜',
 '红薯',
 '莜麦',
 '大麦',
 '水稻',
 '豇豆',
 '刀豆',
 '芸豆',
 '土豆',
 '西红柿',
 '茄子',
 '菠菜 ',
 '青椒',
 '菜花',
 '包菜',
 '油麦菜',
 '小青菜',
 '黄瓜',
 '生菜 ',
 '辣椒',
 '空心菜',
 '黄心菜',
 '芹菜',
 '大白菜',
 '白萝卜',
 '红萝卜',
 '榆黄菇',
 '香菇',
 '白灵菇',
 '羊肚菌']

In [29]:
years = list(range(2024, 2031))  # 2024到2030年
seasons = [1, 2]  # 两个种植季节
crops = list(range(1, 42))  # 41种作物
fields = list(range(1, 55))  # 54块地

In [30]:
# 定义决策变量，X[i][j][k] 表示第 i 作物在第 j 地块的第 k 季度种植面积
X = LpVariable.dicts("X", (crops, fields, seasons, years), lowBound=0, cat='Continuous')

In [ ]:
df3 = pd.read_excel('/home/maxdmx/math-China/附件2(2)1..xlsx')
yields=np.zeros((42,55,3))



In [ ]:
# 目标函数1: 滞销浪费情况下的利润最大化
prob += lpSum([lpSum([lpSum([lpSum(
    [min(yields[c, l, s] * X[c][l][s][y], expected_sales[c]) * prices[c] - costs[c, l, s] * X[c][l][s][y]
     for c in crops] for l in fields]) for s in seasons]) for y in years])

In [ ]:
# 约束条件1: 作物种植面积不能超过地块总面积
field_areas = {l: 55 for l in fields}  
for y in years:
    for s in seasons:
        for l in fields:
            prob += pulp.lpSum([X[c][l][s][y] for c in crops]) <= field_areas[l]

In [ ]:
# 约束条件2: 每个地块三年内至少种一次豆类作物
beans = [1, 2, 3, 4, 5, 17, 18, 19] 
for l in fields:
    for s in seasons:
        for y in years[:-2]:
            prob += pulp.lpSum([X[bean][l][s][y + i] for bean in beans for i in range(3)]) >0


In [ ]:
# 约束条件3: 分散性约束，某个作物不能在太多地块种植
max_field_count = 3  # 每个作物每季最多种4个地块
for y in years:
    for s in seasons:
        for c in crops:
            prob += pulp.lpSum([X[c][l][s][y] > 0 for l in fields]) <= max_field_count


In [ ]:
#约束条件4:种植面积不能太小
#定义B数组
M =10000
m = 0.001
B = LpVariable.dicts("B", (crops, fields, seasons, years), cat = 'Binary')
# 
for c in crops:
    for l in fields:
        for s in seasons:
            for y in years:
                if X[c, l, s, y] > 0:
                    B[c, l, s, y] = 1
                else:
                    B[c, l, s, y] = 0

给B赋值
for c in crops:
    for l in fields:
        for s in seasons:
            for y in years:
                # 约束：如果X[c][l][s][y] > 0，则B[c][l][s][y] 等于1，否则等于0
                prob += X[c, l, s, y] <= M * B[c, l, s, y]
                prob += X[c, l, s, y] >= m * B[c, l, s, y]


In [ ]:
#约束条件5:轮作约束
#对于平旱地
for c in range(1,16):
    for l in range(1,27):
        for y in range(2023,2030):
            prob += X[c, l, 1, y]*X[c, l, 1, y+1] == 0
#对于智慧大棚
for c in range(1,16):
    for l in range(51,55):
        for y in range(2023,2030):
            prob += X[c, l, 1, y]*X[c, l, 1, y+1] +X[c, l, 2, y]*X[c, l, 2, y+1] == 0
            

In [ ]:
#种植类别约束
#对于平旱地，梯田，山坡:
for c in range(1,16):
    for l in range(1,27):
            for y in years:
                prob += X[c, l, 2, y] == 0 

for c in range(17,42):
     for l in range(1,27):
          for s in range(1,3):
            for y in years:
                prob += X[c, l, s, y] == 0
#对于水浇地:
for l in range(27,35):
     for y in years:
          if c>=35 or c<=16:
               prob += X[c,l,1,y]==0
               prob += X[c,l,2,y]==0
#对于普通大棚类:
for l in range(35,51):
     for y in years:
          if c>=35 or c<=16:
               prob += X[c,l,1,y]==0
          if c>=42 or c<=37:
               prob += X[c,l,2,y]==0
#对于智慧大棚类:
for l in range(51,55):
     for s in range(1,3):
        for y in years:
             if c>=35 or c<=16:
                  prob += X[c,l,s,y]==0
          
               
               


